In [1]:
##!pip install pdfminer

In [2]:
import pandas as pd
import re
from datetime import datetime
import io
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [3]:
#ingresar el nombre del documento a analizar
documento = "OVD 3.pdf"

In [4]:
#obtener metadata del documento, ver si sirve
fp = open(documento, 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)

print(doc.info)  

[{'Author': b'pc', 'CreationDate': b"D:20200901203948-03'00'", 'ModDate': b"D:20200901203948-03'00'", 'Producer': b'Microsoft: Print To PDF', 'Title': b'Microsoft Word - Legajo Digital OVD 1'}]


In [5]:
#funcion para convertir el pdf a texto plano
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            
        text = fake_file_handle.getvalue()
    
    # close open handles
    converter.close()
    fake_file_handle.close()
    
    if text:
        return text

In [6]:
# GENERO_ACUSADO/A

In [7]:
#creamos la variable data y le cargamos el texto extraido del pdf
data = extract_text_from_pdf("OVD 4.pdf")

In [8]:
#pasamos el texto a minusculas, y cambiamos los saltos de lineas por espacios
data = data.lower()
data = data.replace('  ', ' ')
data = data.replace('\'', '"').replace('´', '"').replace('‘', '"').replace('’', '"')
print(data[0:100])

   corte suprema de justicia de la nación  oficina de violencia doméstica // la ciudad de buenos air


In [9]:
import unicodedata

def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

data = strip_accents(data)

In [10]:
#como regex es greedy tenemos que aclarar que sea hasta la primera coincidencia con el signo de pregunta.

nrolegajo = re.search(r'informe interdisciplinario de.*?legajo.*?(\d+/\d+)', data)

if nrolegajo == None:
    nrolegajo = "no_encontrado"
else:
    nrolegajo = nrolegajo.groups(1)
nrolegajo

('4407/2020',)

In [11]:
genero_denunciado = re.search(r' relacion a.(.*?) de nacionalidad', data).group(1)
genero_denunciado

'su ex pareja no conviviente el sr. pedro vega, d.n.i. n 44.444.444, nacido el 21 de julio de 1990, de 30 anos de edad,'

In [12]:
findhObj = re.findall ( r'(sr.|sra.)', genero_denunciado)

if 'sr.' in findhObj:
    print ("masculino")
elif 'sra.' in findhObj:
    print ("femenino")
else:
    print ("No match!!")

masculino


In [13]:
info_denunciado= re.search(r' relacion a.(.*?) ingresos', data).group(1)

In [14]:
nacionalidad_denunciado = re.search(r' de nacionalidad (.*?),', info_denunciado).groups(1)
nacionalidad_denunciado

('argentina',)

In [15]:
edaddenunciado = re.search(r', de (.*?) anos de edad', info_denunciado).groups(1)
edaddenunciado

('30',)

In [16]:
instruccion_denunciado = re.search(r'(estudios .*?etos)', info_denunciado).groups(1)
instruccion_denunciado

('estudios secundarios incompletos',)

In [17]:
info_denunciado

'su ex pareja no conviviente el sr. pedro vega, d.n.i. n 44.444.444, nacido el 21 de julio de 1990, de 30 anos de edad, de nacionalidad argentina, de estado civil soltero, domiciliado en el mismo barrio que la senora, cuyo telefono celular refiere desconocer, con nivel de estudios secundarios incompletos, desconociendo su condicion laboral e'

In [18]:
domicilio= re.search(r' barrio (.*?),', info_denunciado).groups(1)
domicilio

('que la senora',)

In [19]:
findhObj = re.findall ( r'(que la senora|que la dicente)', domicilio)

if 'que la senora' in findhObj:
    print (##variable domicilio denunciante)

SyntaxError: unexpected EOF while parsing (<ipython-input-19-f4dfb5033a33>, line 4)